In [4]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm

In [5]:
TRAIN_DATA = [('Equity ETPs\t7.000\t$327.8200\t$2294.74\t$1998.42\t', {'entities': [(12, 17, 'Quantity')]}), ('ISHARES S&P 500(IVV)\t37.000\t24.4900\t906.13\t973.66\t', {'entities': [(16, 19, 'Ticker'), (21, 27, 'Quantity')]}), ('SCHWAB FUNDAMENTAL EM L/C(FNDE) VANGUARD TAX-MANAGED FDS FTSE DEV\t34.000\t39.8100\t1353.54\t1274.05\t', {'entities': [(26, 30, 'Ticker'), (66, 72, 'Quantity')]}), ('MKT ETF (VEA)\t\t\t4554.41\t4246.13\t', {'entities': [(9, 12, 'Ticker')]}), ('Fixed Income ETPs SCHWAB STRATEGIC TR INTERMEDIATE-TERM US\t40.000\t$58.9600\t$2358.40\t$2333.21 1236.03\t', {'entities': [(59, 65, 'Quantity')]}), ('TREASURY ETF (SCHR)\t23.000\t61.3700\t1411.51\t\t', {'entities': [(14, 18, 'Ticker'), (20, 26, 'Quantity')]}), ('VANGUARD BD INDEX FDS SHORT TRM (SCHP)\t28.000\t83.3000\t2332.40\t2238.09 6006.27\t', {'entities': [(33, 37, 'Ticker'), (39, 45, 'Quantity')]}), ('BOND (BSV) VANGUARD BD INDEX FDS TOTAL BND\t73.000\t89.4600\t6530.58\t\t', {'entities': [(6, 9, 'Ticker'), (43, 49, 'Quantity')]}), ('MRKT (BND) Fixed Income ETPs(27% of account holdings)\t\t\t( 12632.89\t11813.60\t', {'entities': [(6, 9, 'Ticker')]}), ('Description Equity ETPs (continued) VANGUARD INDEX FDS VANGUARD SMALL CAP VALUE VIPERS FORMERLY VANGUARD\t14.695\t110.4400\t1622.91\t1774.09 3239.93\t', {'entities': [(105, 111, 'Quantity')]}), ('VANGUARD INDEX FDS VANGUARD MID CAP INDEX TR (VBR) VIPERS FORMERLY VANGUARD INDEX TR\t21.778\t174.2900\t3795.68\t3232.25\t', {'entities': [(46, 49, 'Ticker'), (85, 91, 'Quantity')]}), ('VANGUARD INDEX FDS VANGUARD VALUE (VO) . TR\t30.931\t103.3100\t3195.48\t11360.7\t', {'entities': [(35, 37, 'Ticker'), (44, 50, 'Quantity')]}), ('ETF FORMERLY VANGUARD INDEX (VTV) VANGUARD INDEX FDS VANGUARD TOTAL STK\t86.133\t165.5200\t14256.73\t\t', {'entities': [(29, 32, 'Ticker'), (72, 78, 'Quantity')]}), ('MKT ETF (VTI) Total Equity ETPs (94% of account holdings) Fixed Income ETPs VANGUARD BD INDEX FDS TOTAL BND MRKT (BND)\t17.265\t$89.4600\t52293.97 ( $1544.52\t51326 $143\t', {'entities': [(9, 12, 'Ticker'), (104, 107, 'Ticker'), (119, 125, 'Quantity')]}), ('Equity ETPs\t80.000\t$29.6500\t$2372.00\t$2501.98 1497.49\t', {'entities': [(12, 18, 'Quantity')]}), ('DBX ETF TR XTRACK MSCI EAFE (DBEF)\t49.000\t24.8700\t1218.63\t\t', {'entities': [(29, 33, 'Ticker'), (35, 41, 'Quantity')]}), ('GLOBAL x FDS GLB x MLP ENRG I(MLPX)\t\t60.2600\t1506.50\t1244.45\t', {'entities': [(30, 34, 'Ticker')]}), ('ISHARES MSCI SOUTH KOREA TF(EWY)\t25.000\t28.2500\t2147.00\t1919.56\t', {'entities': [(28, 31, 'Ticker'), (33, 39, 'Quantity')]}), ('ISHARES INC MSCI GERMANY ETF (EWG)\t76.000 10.000\t71.3000\t713.00\t591.37\t', {'entities': [(30, 33, 'Ticker'), (35, 41, 'Quantity')]}), ('ISHARES GLOBAL HEALTHCARE ETF(IXJ)\t\t18.8200\t959.82\t756.00\t', {'entities': [(30, 33, 'Ticker')]}), ('ISHARES GLOBAL ENERGY TF(IXC)\t51.000 116.000\t39.8900\t4627.24\t6016.75\t', {'entities': [(25, 28, 'Ticker'), (30, 36, 'Quantity')]}), ('ISHARES TR EAFE VALUE ETF(EFV)\t132.000\t25.9300\t3422.76\t4470.83\t', {'entities': [(26, 29, 'Ticker')]}), ('ISHARES TR MSCI UK ETF NEW(EWU)\t87.000\t22.6100\t1967.07\t1989.29\t', {'entities': [(27, 30, 'Ticker'), (32, 38, 'Quantity')]}), ('J P MORGAN EXCHANGE-TRADED FD BETBULD JAPAN (BBJP)\t\t27.6900\t1024.53\t1048.28\t', {'entities': [(45, 49, 'Ticker')]}), ('SPDR PORTFOLIO S&P 600 SMALL CAP ETF (SPSM)\t37.000\t24.4900\t1812.26\t2124.22\t', {'entities': [(38, 42, 'Ticker'), (44, 50, 'Quantity')]}), ('SCHWAB FUNDAMENTAL EM L/C(FNDE) VANECK VECTORS ETF TR MORNINGSTAR WIDE (MOAT)\t74.000 14.000\t53.5600 39.8100\t749.84 3622.71\t592.33 4012.26\t', {'entities': [(26, 30, 'Ticker'), (72, 76, 'Ticker'), (78, 84, 'Ticker'), (85, 91, 'Quantity')]}), ('VANGUARD TAX-MANAGED FDS FTSE DEV\t91.000\t.\t\t1569.23\t', {'entities': [(34, 40, 'Quantity')]}), ('MKT ETF (VEA) /ANGUARD WORLD FDS VANGUARD CONSUMER STAPLES ETF (VDC)\t11.000\t159.9000 58.9100\t1758.90 1354.93\t1240.62\t', {'entities': [(9, 12, 'Ticker'), (64, 67, 'Ticker'), (69, 75, 'Quantity')]}), ('ANGUARD SECTOR INDEX FDS VANGUARD\t23.000\t\t\t\t', {'entities': [(34, 40, 'Quantity')]}), ('Description Equity ETPs (continued) VANGUARD INDEX FDS VANGUARD SMALL CAP VIPERS FORMERLY VANGUARD\t65.000\t110.4400\t7178.60\t8228.00\t', {'entities': [(99, 105, 'Quantity')]}), ('VALUE VANGUARD INDEX FDS VANGUARD MID CAP INDEX TR (VBR) VIPERS FORMERLY VANGUARD INDEX TR\t100.000\t174.2900\t17429.00\t15718.56 12505.26\t', {'entities': [(52, 55, 'Ticker'), (91, 98, 'Quantity')]}), ('VANGUARD (VO) INDEX FDS VANGUARD VALUE\t126.000\t103.3100\t13017.06\t53323.07\t', {'entities': [(10, 12, 'Ticker'), (39, 46, 'Quantity')]}), ('ETF FORMERLY VANGUARD INDEX TR (VTV) VANGUARD INDEX FDS VANGUARD TOTAL STK\t375.000\t165.5200\t62070.00 (\t\t', {'entities': [(32, 35, 'Ticker'), (75, 82, 'Quantity')]}), ('MKT ETF (VTI) Total Equity ETPs (60% of account holdings) Fixed Income ETPs\t63.000\t$113.0900\t227087.57 $7124.67\t221000.97 $6622.76\t', {'entities': [(9, 12, 'Ticker'), (76, 82, 'Quantity')]}), ('ISHARES TR JP MOR EM MK ETF (EMB) ISHARES TR BROAD USD HIGH(USHY)\t238.000\t40.0300\t9527.14\t9015.22 7129.35\t', {'entities': [(29, 32, 'Ticker'), (60, 64, 'Ticker'), (66, 73, 'Quantity')]}), ('SCHWAB STRATEGIC TR US TIPS ETF\t129.000\t61.3700\t7916.73\t\t', {'entities': [(32, 39, 'Quantity')]}), ('(SCHP) VANECK VECTORS ETF TR JP MORGAN MKTS (EMLC)\t121.000\t31.5800\t3821.18\t3786.13 14703.21\t', {'entities': [(45, 49, 'Ticker'), (51, 58, 'Quantity')]}), ('VANGUARD BD INDEX FDS SHORT TRM BOND (BSV)\t184.000\t83.3000\t15327.20\ta\t', {'entities': [(38, 41, 'Ticker'), (43, 50, 'Quantity')]}), ('VANGUARD BD INDEX FDS TOTAL BND MRKT (BND)\t741.000\t89.4600\t66289.86\t60332.77\t', {'entities': [(28, 31, 'Ticker'), (43, 50, 'Quantity')]}), ('VANGUARD SCOTTSDALE FDS INTER TERM TREAS (VGIT)\t272.000\t70.8200\t19263.04\t18094.45\t', {'entities': [(42, 46, 'Ticker'), (48, 55, 'Quantity')]}), ('VANGUARD SCOTTSDALE FDS VANGUARD MTG-BACKED SECS IDX FD ETF (VMBS)\t95.000\t54.4100\t5168.95\t5073.63\t', {'entities': [(61, 65, 'Ticker'), (67, 73, 'Quantity')]}), ('VANGUARD SCOTTSDALE FDS INTERMEDIATE TERM CORPORATE BOND INDEX (VCIT)\t115.000\t96.8300\t11135.45\t10443.02\t', {'entities': [(64, 68, 'Ticker'), (70, 77, 'Quantity')]}), ('DBX ETF TR XTRCKR MSCI US (USSG)\t91.000\t$29.5800\t$2691.78\t$2396.03\t$295.75\t$41.68\t1.550%\t', {'entities': [(27, 31, 'Ticker'), (33, 39, 'Quantity')]}), ('ISHARES TR ESG AWRE 1 5 YR(SUSB)\t188.000\t$26.1700\t$4919.96\t$4810.92\t$109.04\t$118.87\t2.420%\t', {'entities': [(27, 31, 'Ticker'), (33, 40, 'Quantity')]}), ('ISHARES TR ESG AWR US AGRGT(EAGG)\t286.000\t57.2000\t16359.20\t15986.63\t372.57\t348.40\t2.130\t', {'entities': [(28, 32, 'Ticker'), (34, 41, 'Quantity')]}), ('SCHWAB STRATEGIC TR SHORT-TERM US TREASURY ETF (SCHO)\t155.000\t51.6300\t8002.65\t8010.23\t-7.58\t142.51\t1.780\t', {'entities': [(48, 52, 'Ticker'), (54, 61, 'Quantity')]}), ('SCHWAB STRATEGIC TR US TIPS ETF (SCHP)\t95.000\t61.3700\t5830.15\t5625.90\t204.25\t-\t- 1.850\t', {'entities': [(33, 37, 'Ticker'), (39, 45, 'Quantity')]}), ('VANGUARD SCOTTSDALE FDS LONG TERM TREAS (VGLT)\t16.000\t104.1400\t1666.24\t1627.12\t39.12\t30.83\t\t', {'entities': [(41, 45, 'Ticker'), (47, 53, 'Quantity')]}), ('Bond Funds (continued) TIAA-CREF CORE IMPACT BOND INSTL (TSBIX)\t1652.285\t11.0800\t18307.31\t17647.11\t660.20\t431.06\t2.350\t', {'entities': [(57, 62, 'Ticker'), (64, 72, 'Quantity')]}), ('ISHARES MSCI SOUTH KOREA ETF(EWY)\t33.000\t$60.2600\t$1988.58\t$1515.02\t$473.56\t', {'entities': [(29, 32, 'Ticker'), (34, 40, 'Quantity')]}), ('ISHARES S&P 500(IVV)\t14.483\t327.8200\t4747.81\t4017.78\t730.03\t', {'entities': [(16, 19, 'Ticker'), (21, 27, 'Quantity')]}), ('ISHARES GLOBAL ENERGY ETF(IXC)\t88.664\t18.8200\t1668.65\t1421.77\t246.88\t', {'entities': [(26, 29, 'Ticker'), (31, 37, 'Quantity')]}), ('ISHARES TR MSCI UK ETF NEW(EWU)\t104.482\t25.9300\t2709.21\t3012.61\t-303.40\t', {'entities': [(27, 30, 'Ticker'), (32, 39, 'Quantity')]}), ('J P MORGAN EXCHANGE-TRADED FD BETBULD JAPAN (BBJP)\t80.489\t22.6100\t1819.85\t1673.90\t145.95\t', {'entities': [(45, 49, 'Ticker'), (51, 57, 'Quantity')]}), ('SCHWAB FUNDAMENTAL EM L/C(FNDE)\t74.970\t24.4900\t1836.01\t1980.05\t-144.04\t', {'entities': [(26, 30, 'Ticker'), (32, 38, 'Quantity')]}), ('VANGUARD TAX-MANAGED FDS FTSE DEV MKT ETF (VEA)\t72.923\t39.8100\t2903.06\t2771.11\t131.95\t', {'entities': [(43, 46, 'Ticker'), (48, 54, 'Quantity')]}), ('VANGUARD SECTOR INDEX FDS VANGUARD FINLS VIPERS (VFH)\t31.223\t58.9100\t1839.34\t1672.74\t166.60\t', {'entities': [(49, 52, 'Ticker'), (54, 60, 'Quantity')]}), ('VANGUARD INDEX FDS VANGUARD MID CAP VIPERS FORMERLY VANGUARD INDEX TR (VO)\t16.474\t174.2900\t2871.25 (\t2410.80\t460.45\t', {'entities': [(71, 73, 'Ticker'), (75, 81, 'Quantity')]}), ('SCHWAB STRATEGIC TR INTERMEDIATE-TERM US TREASURY ETF (SCHR)\t47.239\tA $58.9600\t$2785.21\t$2742.45\t$42.76\t', {'entities': [(55, 59, 'Ticker'), (61, 67, 'Quantity')]}), ('SCHWAB STRATEGIC TR US TIPS ETF (SCHP)\t28.738\t61.3700\t1763.65\t1593.01\t170.64\t', {'entities': [(33, 37, 'Ticker'), (39, 45, 'Quantity')]}), ('VANGUARD BD INDEX FDS SHORT TRM BOND (BSV)\t20.328\t83.3000\t1693.32\t1616.65\t76.67\t', {'entities': [(38, 41, 'Ticker'), (43, 49, 'Quantity')]}), ('VANGUARD BD INDEX FDS TOTAL BND MRKT (BND)\t103.906\t89.4600\t9295.43\t8430.61\t864.82\t', {'entities': [(28, 31, 'Ticker'), (43, 50, 'Quantity')]}), ('ALPHABET INC CAP STK CL C(GOOG)\t3.000 8.000\t1487.9500\t11903.60\t8914.62\t', {'entities': [(26, 30, 'Ticker'), (32, 37, 'Quantity')]}), ('ALPHABET INC CAP STK CL A(GOOGL)\t2.000\t3164.6800\t6329.36\t3259.32\t', {'entities': [(26, 31, 'Ticker'), (33, 38, 'Quantity')]}), ('AMAZON.COM INC (AMZN) AMERISOURCEBERGEN CORPORATION COM\t53.000\t100.1900\t5310.07\t4627.96\t', {'entities': [(16, 20, 'Ticker'), (56, 62, 'Quantity')]}), ('USD0.01 (ABC)\t14.000\t273.8000\t3833.20\t3614.92\t', {'entities': [(9, 12, 'Ticker'), (14, 20, 'Quantity')]}), ('ANTHEM INC (ANTM)\t\t1662.1300\t4986.39\t5408.17\t', {'entities': [(12, 16, 'Ticker'), (19, 28, 'Quantity')]}), ('BOOKING HOLDINGS INC COM(BKNG)\t3.000\t96.9700\t3296.98\t2393.60\t', {'entities': [(25, 29, 'Ticker'), (31, 36, 'Quantity')]}), ('CARMAX INC (KMX) WALT DISNEY co (DIS)\t34.000 8.000\t116.9400\t935.52\t1068.31 5339.44\t', {'entities': [(22, 25, 'Ticker'), (12, 15, 'Ticker'), (38, 44, 'Quantity'), (45, 50, 'Quantity')]}), ('SEDOL #2466149 (ENB) FACEBOOK INC-CLASS A(FB)\t33.000\t253.6700\t6 8371.11\t4960.34 6344.42\t', {'entities': [(0, 3, 'Ticker'), (42, 44, 'Ticker'), (46, 52, 'Quantity'), (53, 61, 'Quantity')]}), ('IQVIA HLDGS INC COM(IQV)\t53.000\t158.3900\t8394.67\t\t', {'entities': [(0, 3, 'Ticker'), (25, 31, 'Quantity')]}), ('LOWES COMPANIES INC COM USD0.50 (LOW)\t\t148.9100\t7743.32\t5167.02 2666.81\t', {'entities': [(0, 3, 'Ticker'), (39, 47, 'Quantity')]}), ('LYFT INC CL A COM (LYFT)\t57.000\t29.2300\t1666.11\t10405.22\t', {'entities': [(0, 4, 'Ticker'), (25, 31, 'Quantity')]}), ('MASTERCARD INCORPORATED CL A(MA)\t52.000\t308.5300\t16043.56\t5194.56\t', {'entities': [(0, 2, 'Ticker'), (33, 39, 'Quantity')]}), ('MICROSOFT CORP (MSFT)\t50.000\t205.0100\t10250.50\t\t', {'entities': [(16, 20, 'Ticker'), (22, 28, 'Quantity')]}), ('TENCENT HOLDINGS LIMITED UNSPON ADR (TCEHY)\t66.000\t68.9010\t4547.46\t2282.28\t', {'entities': [(37, 42, 'Ticker'), (44, 50, 'Quantity')]}), ('UBER TECHNOLOGIES INC COM(UBER)\t77.000\t30.2600\t2330.02\t2501.16 3436.3)\t', {'entities': [(0, 4, 'Ticker'), (32, 38, 'Quantity')]}), ('UNITEDHEALTH GROUP INC(UNH)\t13.000\t302.7800\t3936.14\t4130.2\t', {'entities': [(23, 26, 'Ticker'), (28, 34, 'Quantity')]}), ('VISA INC (V)\t30.000\t190.4000\t5712.00\t\t', {'entities': [(0, 1, 'Ticker'), (13, 19, 'Quantity')]}), ('Description FIDELITY GOVERNMENT CASH RESERVES (FDRXX) -day yield: 0.01%\tQuantity 1594.840\tPer Unit $1.0000\tMarket Value $1594.84\tCost not applicable\tGain/Loss not applicable\tIncome (EAI) $17.73\t(EY) 1.110%\t', {'entities': [(47, 52, 'Ticker'), (81, 89, 'Quantity')]}), ('Stock Funds CALVERT US LRG CAP VAL RESPONS INDEX 1(CFJIX)\t81.466\t$22.4100\t$1825.65\t$1600.00\t$225.65\t$31.99\t1.750%\t', {'entities': [(51, 56, 'Ticker'), (58, 64, 'Quantity')]}), ('DFA INTERNATIONAL SUSTAIN CORE 1 PORT (DFSPX)\t459.580\t10.0500\t4618.77 4\t4028.79\t589.98\t86.53\t1.870\t', {'entities': [(39, 44, 'Ticker'), (46, 53, 'Quantity')]}), ('DOMINI IMPACT INTL EQUITY INSTITUTIONAL (DOMOX)\t248.586\t7.2300\t1797.27\t1561.12\t236.15\t-\t-\t', {'entities': [(41, 46, 'Ticker'), (48, 55, 'Quantity')]}), ('MORNINGSTAR UNCONSTRAINED ALLOC (MSTSX)\t266.075\t10.0600\t2676.71\t2400.00\t276.71\t56.62\t2.120\t', {'entities': [(33, 38, 'Ticker'), (40, 47, 'Quantity')]}), ('MORNINGSTAR ALTERNATIVES INSTL (MSTVX)\t1240.157\t10.5400\t13071.25\t12600.00\t471.25\t179.09\t1.370\t', {'entities': [(32, 37, 'Ticker'), (39, 47, 'Quantity')]}), ('PARNASSUS MID CAP FD INSTITUTIONAL (PFPMX)\t54.164\t34.9200\t1891.40\t1600.00\t291.40\t13.86\t0.730\t', {'entities': [(36, 41, 'Ticker'), (43, 49, 'Quantity')]}), ('PARNASSUS CORE EQUITY INSTL(PRILX)\t57.012\t49.0900\t2798.71\t2405.21\t393.50\t25.92\t0.930\t', {'entities': [(28, 33, 'Ticker'), (35, 41, 'Quantity')]}), ('RBC EMERGING MARKETS EQUITY FD CL I (REEIX)\t156.556\t12.2400\t1916.24\t1600.00\t316.24\t59.18\t3.090\t', {'entities': [(37, 42, 'Ticker'), (44, 51, 'Quantity')]}), ('AMERICAN BEACON GL EV FRNT MRKT INC Y (AGEYX)\t536.966\t$8.2000\t$4403.12\t$4056.77\t$346.35\t$380.17\t8.630%\t', {'entities': [(39, 44, 'Ticker'), (46, 53, 'Quantity')]}), ('CALVERT HIGH YIELD BOND CLASS (CYBIX)\t65.477\t26.4600\t1732.52\t1609.10\t123.42\t84.40\t4.870\t', {'entities': [(31, 36, 'Ticker'), (38, 44, 'Quantity')]}), ('COMMUNITY CAPITAL TRUST INSTL (CRANX)\t370.072\t10.9000\t4033.78\t4015.29\t18.49\t100.76\t2.500\t', {'entities': [(31, 36, 'Ticker'), (38, 45, 'Quantity')]}), ('MORNINGSTAR DEFENSIVE BOND INSTL (MSTBX)\t452.233\t10.3100\t4662.52\t4612.86\t49.66\t101.99\t2.190\t', {'entities': [(34, 39, 'Ticker'), (41, 48, 'Quantity')]}), ('ISHARES GLOBAL ENERGY ETF(IXC)\t31.000\t$18.8200\t$583.42 $4\t', {'entities': [(26, 29, 'Ticker'), (31, 37, 'Quantity')]}), ('ISHARES TR EAFE ETF(EFV)\t22.000\t39.8900\t877.58 7\t', {'entities': [(20, 23, 'Ticker'), (25, 31, 'Quantity')]}), ('ISHARES TR MSCI UK ETF NEW (EWU)\t25.000\t25.9300\t648.25 5\t', {'entities': [(28, 31, 'Ticker'), (33, 39, 'Quantity')]}), ('J P MORGAN EXCHANGE-TRADED FD BETBULD JAPAN (BBJP)\t29.000\t22.6100\t655.69 5\t', {'entities': [(45, 49, 'Ticker'), (51, 57, 'Quantity')]}), ('SCHWAB STRATEGIC TR US MID-CAP ETF (SCHM)\t23.000\t55.3000\t1271.90 1\t', {'entities': [(36, 40, 'Ticker'), (42, 48, 'Quantity')]}), ('SCHWAB FUNDAMENTAL EM L/C(FNDE)\t28.000\t24.4900\t685.72 1\t', {'entities': [(26, 30, 'Ticker'), (32, 38, 'Quantity')]}), ('MKT ETF (VEA) VANGUARD INTL EQUITY INDEX FDS FTSE EMR\t17.000 .\t43.0100\t731.17 1\t', {'entities': [(9, 12, 'Ticker'), (54, 60, 'Quantity')]}), ('MKT ETF (VWO) VANGUARD INDEX FDS VANGUARD VALUE INDEX TR (VTV)\t15.000\t103.3100\t1549.65 4138.00 3\t', {'entities': [(9, 12, 'Ticker'), (58, 61, 'Ticker'), (63, 69, 'Quantity'), (70, 78, 'Quantity')]}), ('MKT ETF (VTI) Total Equity ETPs (20% of account holdings) Fixed Income ETPs JP MOR EM MK ETF (EMB)\t21.000\t$113.0900\t12733.78 10 $2374.89 $2\t', {'entities': [(9, 12, 'Ticker'), (94, 97, 'Ticker'), (99, 105, 'Quantity')]}), ('ISHARES MSCI SOUTH KOREA TF(EWY)\t53.000\t$60.2600 327.8200\t7539.86\t6618.71\t921.15\t', {'entities': [(28, 31, 'Ticker'), (33, 39, 'Quantity')]}), ('ISHARES S&P 500(IVV)\t23.000\t18.8200\t2615.98\t2184.71\t431.27\t', {'entities': [(16, 19, 'Ticker'), (21, 27, 'Quantity')]}), ('ISHARES GLOBAL ENERGY ETF(IXC)\t139.000\t25.9300\t4304.38\t5029.55\t-725.17\t', {'entities': [(26, 29, 'Ticker'), (31, 38, 'Quantity')]}), ('ISHARES TR MSCI UK ETF NEW (EWU) J P MORGAN EXCHANGE-TRADED FD\t166.000 128.000\t22.6100\t2894.08\t2634.37\t259.71 -388.91\t', {'entities': [(28, 31, 'Ticker'), (63, 70, 'Quantity')]}), ('BETBULD JAPAN (BBJP)\t125.000\t24.4900\t3061.25\t3450.16\t\t', {'entities': [(15, 19, 'Ticker'), (21, 28, 'Quantity')]}), ('SCHWAB FUNDAMENTAL EM L/C(FNDE) VANGUARD TAX-MANAGED FDS FTSE DEV\t114.000\t39.8100\t4538.34\t4589.96 2677.50\t-51.62 268.00\t', {'entities': [(26, 30, 'Ticker'), (66, 73, 'Quantity')]}), ('MKT ETF (VEA) VANGUARD SECTOR INDEX FDS VANGUARD\t50.000\t58.9100\t2945.50\t4390.65\t315.18\t', {'entities': [(49, 55, 'Quantity')]}), ('FINLS VIPERS (VFH) VANGUARD INDEX FDS VANGUARD MID CAP VIPERS FORMERLY VANGUARD INDEX TR\t27.000\t174.2900\t4705.83\t\t\t', {'entities': [(14, 17, 'Ticker'), (89, 95, 'Quantity')]}), ('TREASURY ETF (SCHR)\t.\t\t\t2540.77\t282.25\t', {'entities': [(14, 18, 'Ticker'), (24, 31, 'Quantity')]}), ('(SCHP) VANGUARD BD INDEX FDS SHORT TRM BOND (BSV)\t33.000\t83.3000\t2748.90\t2593.85\t1538.24\t', {'entities': [(1, 5, 'Ticker'), (45, 48, 'Ticker'), (50, 56, 'Quantity'), (57, 64, 'Quantity')]}), ('VANGUARD BD INDEX FDS TOTAL BND MRKT (BND)\t164.000\t89.4600\t14671.44\t13133.20 22265.38\t2046.22\t', {'entities': [(28, 31, 'Ticker'), (43, 50, 'Quantity')]}), ('UNILEVER PLC SPON ADR NEW(UL)\t151.000\t60.4800\t9132.48\t', {'entities': [(26, 28, 'Ticker'), (30, 37, 'Quantity')]}), ('VISA INC (V)\t34.000\t190.4000\t6473.60\t', {'entities': [(0, 1, 'Ticker'), (13, 19, 'Quantity')]}), ('WPP PLC AMERICAN DEPOSITARY SHS EACH REP 5 ORD (WPP)\t32.000\t37.3000\t1193.60\t', {'entities': [(0, 3, 'Ticker'), (53, 59, 'Quantity')]}), ('WELLS FARGO co NEW COM(WFC)\t156.000\t24.2600\t3784.56\t', {'entities': [(23, 26, 'Ticker'), (28, 35, 'Quantity')]}), ('ALTRIA GROUP INC(MO)\t28.000\t$41.1500\t$1152.20\t$1823.26\t-$671.06\t$96.\t', {'entities': [(17, 19, 'Ticker'), (21, 27, 'Quantity')]}), ('AMGEN INC (AMGN)\t15.000\t244.6700\t3670.05\t2767.32\t902.73\t96\t', {'entities': [(11, 15, 'Ticker'), (17, 23, 'Quantity')]}), ('BLACKROCK INC COM USD0.01(BLK)\t5.000\t575.0100\t2875.05\t2077.14\t797.91\t72\t', {'entities': [(26, 29, 'Ticker'), (31, 36, 'Quantity')]}), ('CATERPILLAR INC COM USD1.00(CAT)\t17.000\t132.8800\t2258.96\t2015.58\t243.38\t70\t', {'entities': [(0, 3, 'Ticker'), (33, 39, 'Quantity')]}), ('CENTERPOINT ENERGY INC COM STK USD0.01 (CNP)\t107.000\t19.0100\t2034.07\t2872.10\t-838.03\t64\t', {'entities': [(40, 43, 'Ticker'), (45, 52, 'Quantity')]}), ('COCA-COLA co (KO)\t65.000\t47.2400\t3070.60\t2861.15\t209.45\t106\t', {'entities': [(14, 16, 'Ticker'), (18, 24, 'Quantity')]}), ('COMCAST CORP (CMCSA)\t90.000\t42.8000\t3852.00\t3393.04\t458.96\t82\t', {'entities': [(14, 19, 'Ticker'), (21, 27, 'Quantity')]}), ('COMPASS MINERALS INTL INC(CMP)\t43.000\t50.9400\t& 2190.42\t2854.42\t-664.00\t123\t', {'entities': [(26, 29, 'Ticker'), (31, 37, 'Quantity')]}), ('DOMINION ENERGY INC COM(D)\t36.000\t81.0300\t2917.08\t2541.92\t375.16\t13\t', {'entities': [(0, 1, 'Ticker'), (27, 33, 'Quantity')]}), ('ISIN #US26441C2044 SEDOL (DUK) ENBRIDGE INC COM ISIN #CA29250N1050 SEDOL #2466149 (ENB)\t156.000\t32.0000\t4992.00\t5597.75\t-605.75\t17\t', {'entities': [(26, 29, 'Ticker'), (31, 34, 'Ticker'), (88, 95, 'Quantity'), (96, 103, 'Quantity')]}), ('EXXON MOBIL CORP(XOM)\t49.000\t42.0800\t2061.92\t3760.82\t-1698.90\t8\t', {'entities': [(17, 20, 'Ticker'), (22, 28, 'Quantity')]}), ('FIRSTENERGY CORP COM USD0.10(FE)\t52.000\t29.0000\t1508.00\t2092.91\t-584.91\t9\t', {'entities': [(29, 31, 'Ticker'), (33, 39, 'Quantity')]}), ('GENERAL MILLS INC COM USD0.10(GIS)\t51.000\t63.2700\t3226.77\t2528.78\t697.99\t\t', {'entities': [(30, 33, 'Ticker'), (35, 41, 'Quantity')]}), ('GENUINE PARTS co COM USD1.00(GPC)\t21.000\t90.1500\t1893.15\t1844.10\t49.05\t1:\t', {'entities': [(29, 32, 'Ticker'), (34, 40, 'Quantity')]}), ('IANESBRANDS INC COM USD0.01(HBI)\t223.000\t14.1300\t3150.99\t4212.30\t-1061.31 835.48\t\t', {'entities': [(28, 31, 'Ticker'), (33, 40, 'Quantity')]}), ('IOME DEPOT INC(HD)\t8.000\t265.4900\t2123.92\t1288.44\t550.43\t1\t', {'entities': [(15, 17, 'Ticker'), (19, 24, 'Quantity')]}), ('LOYDS BANKING GROUP ADR 4 ORD(LYG)\t1245.000\t1.3200\t1643.40\t2904.58\t273.45\t\t', {'entities': [(30, 33, 'Ticker'), (35, 43, 'Quantity')]}), ('ICDONALD S CORP(MCD)\t6.000\t194.2800\t1165.68\t892.23\t-586.74\t\t', {'entities': [(16, 19, 'Ticker'), (21, 26, 'Quantity')]}), ('MNICOM GROUP INC COM USD0.15(OMC)\t26.000\t53.7300\t1396.98\t1983.72\t424.73\t\t', {'entities': [(29, 32, 'Ticker'), (34, 40, 'Quantity')]}), ('FIZER INC (PFE)\t139.000\t38.4800\t5348.72\t4923.99\t\t\t', {'entities': [(11, 14, 'Ticker'), (16, 23, 'Quantity')]}), ('FIDELITY TREASURY MONEY MARKET FUND (FZFXX) 7-day vield: 0.01%\t7021.340\t$1.0000\t$7021.34\tnot applicable\tnot applicable\t$74.86\t', {'entities': [(37, 42, 'Ticker'), (63, 71, 'Quantity')]}), ('ISHARES CORE S&P 500 ETF(IVV)\t6.000\t$309.6900\t$1858.14\t$1632.30\t$225.84\t$35.26\t1.900%\t', {'entities': [(25, 28, 'Ticker'), (30, 35, 'Quantity')]}), ('SCHWAB FUNDAMENTAL EM L/C(FNDE)\t41.000\t23.4100\t959.81\t1084.28\t-124.47\t20.37\t2.120\t', {'entities': [(26, 30, 'Ticker'), (32, 38, 'Quantity')]}), ('VANGUARD TAX-MANAGED FDS FTSE DEV MKT ETF (VEA)\t38.000\t38.7900\t1474.02\t1430.84\t43.18\t39.69\t2.690\t', {'entities': [(43, 46, 'Ticker'), (48, 54, 'Quantity')]}), ('ISHARES TR JP MOR EM MK ETF (EMB)\t9.000\t$109.2200\t$982.98\t$910.62\t$72.36\t$43.88\t4.460%\t', {'entities': [(29, 32, 'Ticker'), (34, 39, 'Quantity')]}), ('SCHWAB STRATEGIC TR INTERMEDIATE-TERN US TREASURY ETF (SCHR)\t42.000\t58.7800\t4 2468.76\t2447.97\t20.79\t49.95\t2.020\t', {'entities': [(55, 59, 'Ticker'), (61, 67, 'Quantity')]}), ('SCHWAB STRATEGIC TR US TIPS ETF (SCHP)\t25.000\t60.0200\t1500.50\t1342.62\t157.88\t-\t-\t', {'entities': [(33, 37, 'Ticker'), (39, 45, 'Quantity')]}), ('VANGUARD BD INDEX FDS SHORT TRM BOND (BSV)\t24.000\t83.1200\t1994.88\t1897.17\t97.71\t41.80\t2.100\t', {'entities': [(38, 41, 'Ticker'), (43, 49, 'Quantity')]}), ('VANGUARD BD INDEX FDS TOTAL BND MRKT (BND)\t72.000\t88.3400\t6360.48\t5873.99\t486.49\t155.99\t2.450\t', {'entities': [(28, 31, 'Ticker'), (43, 49, 'Quantity')]}), ('ETF TR XTRACK MSCI EAFE (DBEF)\t186.000\t\t', {'entities': [(25, 29, 'Ticker'), (31, 38, 'Quantity')]}), ('x FDS GLB x MLP ENRG I(MLPX)\t113.000\t\t', {'entities': [(23, 27, 'Ticker'), (29, 36, 'Quantity')]}), ('INC MSCI GERMANY ETF (EWG)\t103.000\t\t', {'entities': [(22, 25, 'Ticker'), (27, 34, 'Quantity')]}), ('GLOBAL ENERGY ETF(IXC)\t143.000\t\t', {'entities': [(18, 21, 'Ticker'), (23, 30, 'Quantity')]}), ('TR EAFE VALUE ETF(EFV)\t245.000\t\t', {'entities': [(18, 21, 'Ticker'), (23, 30, 'Quantity')]}), ('TR MSCI UK ETF NEW (EWU)\t218.000\t\t', {'entities': [(20, 23, 'Ticker'), (25, 32, 'Quantity')]}), ('MORGAN EXCHANGE-TRADED FD JAPAN (BBJP)\t182.000\t\t', {'entities': [(33, 37, 'Ticker'), (39, 46, 'Quantity')]}), ('FUNDAMENTAL EM L/C(FNDE)\t178.000\t2\t', {'entities': [(19, 23, 'Ticker'), (25, 32, 'Quantity')]}), ('VECTORS ETF TR MORNINGSTAR (MOAT)\t54.000\t5\t', {'entities': [(28, 32, 'Ticker'), (34, 40, 'Quantity')]}), ('TAX-MANAGED FDS FTSE DEV ETF (VEA)\t217.000\t39\t', {'entities': [(30, 33, 'Ticker'), (35, 42, 'Quantity')]}), ('SECTOR INDEX FDS VANGUARD VIPERS (VFH)\t49.000\t58\t', {'entities': [(34, 37, 'Ticker'), (39, 45, 'Quantity')]}), ('INTL EQUITY INDEX FDS FTSE ETF (VWO)\t176.000\t43\t', {'entities': [(32, 35, 'Ticker'), (37, 44, 'Quantity')]}), ('Equity ETPs (continued) VANGUARD TAX-MANAGED FDS FTSE DEV MKT ETF (VEA)\t109.000\t39.8100\t4339.29\t4103.05\t236.24\t\t', {'entities': [(67, 70, 'Ticker'), (72, 79, 'Quantity')]}), ('ISHARES SHORT-TERM NATIONAL MUNI BOND ETF (SUB)\t116.000\t$108.3900\t$12573.24\t$12270.85\t$302.39\t\t', {'entities': [(43, 46, 'Ticker'), (48, 55, 'Quantity')]}), ('VANGUARD MUN BD FDS TAX EXEMPT BD (VTEB)\t362.000\t54.9000\t19873.80 6\t18976.18\t897.62\t\t', {'entities': [(35, 39, 'Ticker'), (41, 48, 'Quantity')]}), ('DBX ETF TR XTRACK MSCI EAFE (DBEF)\t381.000\t$29.6500\t$11296.65\t', {'entities': [(29, 33, 'Ticker'), (35, 42, 'Quantity')]})]
     

In [6]:
model = None
output_dir=Path("../Model")
n_iter=100

In [7]:

#load the model

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

Created blank 'en' model


In [6]:
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            nlp.update(
                [text],  
                [annotations],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

/Users/Nagarwa/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/spacy/language.py:639: UserWarning: [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the model to some degree. If this is intentional or the language you're using doesn't have a normalization table, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed. The languages with lexeme normalization tables are currently: da, de, el, en, id, lb, pt, ru, sr, ta, th.
  **kwargs
  0%|          | 0/161 [00:00<?, ?it/s]/Users/Nagarwa/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "IOME DEPOT INC(HD)	8.000	265.4900	2123.92	1288.44	..." with entities "[(15, 17, 'Ticker'), (19, 24, 'Quantity')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligne

 22%|██▏       | 35/161 [00:00<00:02, 52.03it/s]/Users/Nagarwa/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "LOWES COMPANIES INC COM USD0.50 (LOW)		148.9100	77..." with entities "[(0, 3, 'Ticker'), (39, 47, 'Quantity')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/Users/Nagarwa/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "BOOKING HOLDINGS INC COM(BKNG)	3.000	96.9700	3296...." with entities "[(25, 29, 'Ticker'), (31, 36, 'Quantity')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/Users/Nagarwa/opt/a

 50%|████▉     | 80/161 [00:01<00:01, 41.38it/s]/Users/Nagarwa/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "SCHWAB FUNDAMENTAL EM L/C(FNDE)	41.000	23.4100	959..." with entities "[(26, 30, 'Ticker'), (32, 38, 'Quantity')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/Users/Nagarwa/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "SEDOL #2466149 (ENB) FACEBOOK INC-CLASS A(FB)	33.0..." with entities "[(0, 3, 'Ticker'), (42, 44, 'Ticker'), (46, 52, 'Q...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
/Users/

 80%|████████  | 129/161 [00:02<00:00, 47.78it/s]/Users/Nagarwa/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "GENERAL MILLS INC COM USD0.10(GIS)	51.000	63.2700	..." with entities "[(30, 33, 'Ticker'), (35, 41, 'Quantity')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
 84%|████████▍ | 135/161 [00:02<00:00, 47.98it/s]/Users/Nagarwa/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "GENUINE PARTS co COM USD1.00(GPC)	21.000	90.1500	1..." with entities "[(29, 32, 'Ticker'), (34, 40, 'Quantity')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.


{'ner': 416.94835379618235}


...

{'ner': 2.0019866562893815}


  2%|▏         | 3/161 [00:00<00:06, 24.91it/s]

{'ner': 1.7049108757148603}


100%|██████████| 161/161 [00:06<00:00, 24.83it/s]

{'ner': 3.6390739679341118}


In [7]:

for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('13.000', 'Quantity')]
Entities [('VTI', 'Ticker'), ('63.000', 'Quantity')]
Entities [('VTV', 'Ticker'), ('86.133', 'Quantity')]
Entities [('NEW(EWU', 'Ticker'), ('87.000', 'Quantity')]
Entities [('DUK', 'Ticker'), ('156.000', 'Quantity'), ('32.0000', 'Quantity')]
Entities [('BBJP', 'Ticker'), ('80.489', 'Quantity')]
Entities [('3.000', 'Quantity')]
Entities [('VO', 'Ticker'), ('30.931', 'Quantity')]
Entities [('SUB', 'Ticker'), ('116.000', 'Quantity')]
Entities [('PFE', 'Ticker'), ('139.000', 'Quantity')]
Entities [('VMBS', 'Ticker'), ('95.000', 'Quantity')]
Entities [('VTEB', 'Ticker'), ('362.000', 'Quantity')]
Entities [('VEA', 'Ticker'), ('38.000', 'Quantity')]
Entities [('SCHM', 'Ticker'), ('23.000', 'Quantity')]
Entities [('57.012', 'Quantity')]
Entities [('LYFT', 'Ticker'), ('57.000', 'Quantity')]
Entities [('CRANX', 'Ticker'), ('370.072', 'Quantity')]
Entities [('6.000', 'Quantity')]
Entities [('VCIT', 'Ticker'), ('115.000', 'Quantity')]
Entities [('BBJP', 'Ticker')]

In [8]:

if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to /Users/Nagarwa/AI/NLP/Model


In [9]:
with open('rawText.txt', 'r') as file:
    data = file.read()

In [10]:
print("Loading from", output_dir)
for line in data.splitlines():
    doc = nlp(line)
    for ent in doc.ents:
        print(ent.label_, ent.text)

Loading from /Users/Nagarwa/AI/NLP/Model
Quantity 7.000
Quantity 37.000
Quantity 34.000
Ticker VEA
Quantity 40.000
Ticker SCHR
Quantity 23.000
Ticker SCHP
Quantity 28.000
Ticker BSV
Quantity 73.000
Ticker BND
Quantity 14.695
Ticker VBR
Quantity 21.778
Ticker VO
Quantity 30.931
Ticker VTV
Quantity 86.133
Ticker VTI
Ticker BND
Quantity 17.265
Quantity 80.000
Ticker DBEF
Quantity 49.000
Quantity 25.000
Ticker EWG
Quantity 76.000
Quantity 51.000
Ticker NEW(EWU
Quantity 87.000
Ticker BBJP
Ticker SPSM
Quantity 37.000
Ticker MOAT
Ticker 74.000
Quantity 14.000
Quantity 91.000
Ticker VEA
Ticker VDC
Quantity 11.000
Quantity 23.000
Quantity 65.000
Ticker VBR
Quantity 100.000
Ticker VO
Quantity 126.000
Ticker VTV
Quantity 375.000
Ticker VTI
Quantity 63.000
Ticker EMB
Quantity 238.000
Quantity 129.000
Ticker EMLC
Quantity 121.000
Ticker BSV
Quantity 184.000
Ticker BND
Quantity 741.000
Ticker VGIT
Quantity 272.000
Ticker VMBS
Quantity 95.000
Ticker VCIT
Quantity 115.000
Ticker EY
Ticker USSG
Quantit